<a href="https://colab.research.google.com/github/timschott/gpt-2/blob/master/gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Going to use https://www.gwern.net/GPT-2 to implement GPT-2. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
 ## Clean up then Clone the repo
#%pwd
## %cd /content
#%cd ..
!git clone https://github.com/timschott/gpt-2
# %cd ../
#%cd ..
#%rm -rf gpt-2/
#%ls
#%cd gpt-2/
#%ls
## can git pull!
#!git pull
## %rm -rf data/ginsburg-lines-v001.txt.npz

Cloning into 'gpt-2'...
remote: Enumerating objects: 529, done.
remote: Total 529 (delta 0), reused 0 (delta 0), pack-reused 529
Receiving objects: 100% (529/529), 4.44 MiB | 6.20 MiB/s, done.
Resolving deltas: 100% (313/313), done.


In [3]:
## Install stuff
%tensorflow_version 1.x
## !python --version
## Kind of nice that all this stuff is already installed. 
## !pip list

TensorFlow 1.x selected.


In [ ]:
## make sure we have a gpu running
## Navigate to Edit→Notebook Settings
## select GPU from the Hardware Accelerator drop-down
!nvidia-smi

In [4]:
# make sure we are mounted correctly
!ls
## cd
%cd gpt-2/

drive  gpt-2  sample_data
/content/gpt-2


In [ ]:
## download the stock model (perhaps move it inside gpt-2 if you're not inside git repo)
!python3 download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 1.10Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 61.3Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.02Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:06, 75.0Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 4.87Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 55.8Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 55.3Mit/s]                                                       


In [ ]:
## make a data container
!mkdir ../data

In [ ]:
#!cd gpt-2/
# for some reason need this percent sign
# %cd gpt-2/
%ls

CONTRIBUTORS.md  download_model.py  models/           train-horovod.py
DEVELOPERS.md    encode.py*         README.md         train.py*
Dockerfile.cpu   __init__.py        requirements.txt
Dockerfile.gpu   LICENSE            src/


In [ ]:
% cat ../drive/My\ Drive/sents/opinions_all.txt | head

In [ ]:
## encode the sentences into the expected format

# % cd gpt-2/
# !git pull

!PYTHONPATH=src ./encode.py ../drive/My\ Drive/sents/opinions_all.txt ../data/opinions-lines-v001.txt.npz

2020-08-16 19:40:33.168176: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Reading files
100% 1/1 [00:19<00:00, 19.09s/it]
Writing ../data/opinions-lines-v001.txt.npz


In [ ]:
import tensorflow
print(tensorflow.__version__)

!pip install toposort
!pip install hparams

1.15.2


In [ ]:
%ls -ll
#!chmod +x train.py


In [ ]:
%rm -rf checkpoint
%rm -rf samples/

In [ ]:
## get latest code and fine tune
!git pull

## fine tune the model
## going to run for a shorter amount of time. i don't want the loss to drop below like 2. 
!PYTHONPATH=src ./train.py --model_name 117M --dataset ../data/opinions-lines-v001.txt.npz \
--batch_size 2 --save_every 100 --sample_every 100 \
--learning_rate=.003 --run_name scalia_1 --top_p .9

In [ ]:
ls checkpoint/

scalia_1/


In [ ]:
mkdir ../drive/My\ Drive/sbotus_output

In [ ]:
%cp -avr checkpoint/ ../drive/My\ Drive/sbotus_output

In [ ]:
%cp -avr ../data/ ../drive/My\ Drive/sbotus_output

In [ ]:
%cp -avr models/117M/ ../drive/My\ Drive/sbotus_output

Post model genration and saving, I am going to load my model from storage and run some text generation scripts with it

In [ ]:
## reclone the repo.
!git clone https://github.com/timschott/gpt-2


In [5]:
## make a models directory to put it in. 
## %cd gpt-2
## make a
%mkdir models

In [ ]:
## reverse the cp's 
%cp -avr ../drive/My\ Drive/sbotus_output_two/117M models/117M/

In [ ]:
%cp -avr ../drive/My\ Drive/sbotus_output_two/checkpoint/scalia_1 models/all_justices_1/

In [8]:
## copy over json / param files from base model to custom (cf https://www.gwern.net/GPT-2#gpt-2-poetry-samples)
%cp models/117M/encoder.json models/all_justices_1/
%cp models/117M/hparams.json models/all_justices_1/
%cp models/117M/vocab.bpe models/all_justices_1/


In [ ]:
## first install fire module.
## !pip install fire 

!git pull

In [17]:
mkdir ../data

In [ ]:
## cook it up
!python src/generate_unconditional_samples.py --top_k 40 --temperature 0.9 --nsamples 100 \
--seed 2 --model_name all_justices_1 --top_p 0.9 > ../data/one_hundred_samples.txt

In [ ]:
mkdir ../drive/My\ Drive/sbotus_output_two

In [ ]:
%cp -avr checkpoint/ ../drive/My\ Drive/sbotus_output_two

In [ ]:
%cp -avr ../data/ ../drive/My\ Drive/sbotus_output_two

'../data/' -> '../drive/My Drive/sbotus_output_two/data'
'../data/opinions-lines-v001.txt.npz' -> '../drive/My Drive/sbotus_output_two/data/opinions-lines-v001.txt.npz'


In [ ]:
%cp -avr models/117M/ ../drive/My\ Drive/sbotus_output_two